In [1]:
import math

import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from numpy import sqrt
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

/Users/tomas.dallas/itba/binance-bot/venv/lib/python3.8/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS = 1000
# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)

df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)
df['ret'] = (df['close'].shift(-1) - df['close']) / df['close']
df['ret'] = df['ret'].apply(lambda x: np.sign(x) if np.abs(x) < 0.0001 else (-2 if np.sign(x) < 0 else 2))


def perc_dif(actual, prediction):
    return np.abs(prediction - actual) / prediction

In [3]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret_h,ret_l,ret
0,1661040000000,21140.07,21163.88,21136.70,21162.06,298.59140,NaN,NaN,NaN,NaN,-2.0
1,1661040180000,21162.06,21170.01,21119.90,21128.62,383.42144,-0.001580,1.040205e-03,0.000290,-0.000795,2.0
2,1661040360000,21128.62,21164.99,21118.71,21163.56,319.63957,0.001654,-1.580186e-03,-0.000237,-0.000056,2.0
3,1661040540000,21163.56,21205.00,21161.60,21189.24,385.29906,0.001213,1.653681e-03,0.001890,0.002031,-2.0
4,1661040720000,21189.24,21190.74,21158.10,21168.10,263.47068,-0.000998,1.213406e-03,-0.000672,-0.000165,2.0
...,...,...,...,...,...,...,...,...,...,...,...
5247,1661984460000,20136.67,20160.39,20134.47,20137.42,523.00044,0.000037,4.966067e-07,0.000608,0.001916,-2.0
5248,1661984640000,20138.89,20156.64,20105.75,20113.18,601.23214,-0.001204,1.102466e-04,-0.000186,-0.001426,2.0
5249,1661984820000,20113.18,20136.82,20110.00,20115.62,376.35440,0.000121,-1.276634e-03,-0.000983,0.000211,-2.0
5250,1661985000000,20114.57,20126.16,20085.70,20103.86,615.45862,-0.000585,6.910891e-05,-0.000529,-0.001208,2.0


In [4]:
hit_df = pd.read_csv("ret_h/oc/errors_knn_oc_3m_normalized.csv")
hit_df.columns = ['hit_rate', 'NDTR', 'predictions']

fig = make_subplots()

fig.add_trace(
    go.Scatter(x=hit_df['NDTR'], y=hit_df['hit_rate'], mode="markers+lines"))

fig.update_layout(yaxis_tickformat=".2%", title="Hit rate of prediction for a given NDTR/K", xaxis_title="NDTR/K",
                  yaxis_title="Hit rate", xaxis_tickformat="none")
fig.show()

ValueError: Length mismatch: Expected axis has 4 elements, new values have 3 elements

In [ ]:
# buys = [0] * NDTR
# sells = [0] * NDTR
# for i in np.arange(NDTR, len(df)):
#     small_df = df.iloc[i: i + 3].copy(deep=True)[['ret', 'ret_p']]
#     if len(small_df) == 3:
#         first = small_df['ret'].iloc[0]
#         second = small_df['ret'].iloc[1]
#         third = small_df['ret_p'].iloc[2]
#         if first < 0 and second < 0 and third > 0.0001:
#             buys.append(1)
#         else:
#             buys.append(0)
#     else:
#         buys.append(0)
# df['buy'] = buys

In [ ]:
# df['sell'] = 0
# for i in np.arange(NDTR, len(df)):
#     small_df = df[['buy', 'close', 'high']]
#     buy = small_df['buy'].iloc[i]
#     if buy == 1:
#         buy_price = small_df['high'].iloc[i]
#         for j in np.arange(i + 1, len(df)):
#             sell_price = small_df['close'].iloc[j]
#             actual_ret = ret(buy_price, sell_price)
#             if actual_ret > 0.0001:
#                 df.at[j, 'sell'] = 1
#                 break

In [ ]:
df